In [11]:
library(dplyr)
library(qvalue)

In [22]:
#### DA day 30
filename0 = "/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/pool1_13_noddd_D30/qtl_analysis/eqtl_discovery/celltype_DA/summary_stats/pool1_13_noddd_D30.DA.qtl_results_all.txt"
#### DA day 52 (untreated)
filename1 = "/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/pool1_17_D52/qtl_analysis/eqtl_discovery_separated_by_treatment/input_files_untreated/celltype_DA/summary_stats/pool1_17_D52.DA.qtl_results_all.txt"
#### DA day 52 rotenone-treated
filename2 = "/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/pool1_17_D52/qtl_analysis/eqtl_discovery_separated_by_treatment/input_files_ROT_treated/celltype_DA/summary_stats/pool1_17_D52.DA.qtl_results_all.txt"

In [25]:
# load full list of results DA conditions (day30, day52, day52+ROT)
# res.day30 = read.csv(filename0, sep="\t")
# res.day52 = read.csv(filename1, sep="\t")
# res.day52rot = read.csv(filename2, sep="\t")

In [26]:
# load lead variant results DA conditions (day30, day52, day52+ROT)
leads.day30 = read.csv(gsub("summary_stats/pool1_13_noddd_D30.DA.","top_",filename0), sep="\t")
leads.day52 = read.csv(gsub("summary_stats/pool1_17_D52.DA.","top_",filename1), sep="\t")
leads.day52rot = read.csv(gsub("summary_stats/pool1_17_D52.DA.","top_",filename2), sep="\t")

In [27]:
leads.day30$global_corr_p_value = qvalue(leads.day30$empirical_feature_p_value)$qvalues
leads.day52$global_corr_p_value = qvalue(leads.day52$empirical_feature_p_value)$qvalues
leads.day52rot$global_corr_p_value = qvalue(leads.day52rot$empirical_feature_p_value)$qvalues

In [109]:
T=0.05

In [110]:
leads.d30.sign = leads.day30[leads.day30$global_corr_p_value < T,]
leads.d52.sign = leads.day52[leads.day52$global_corr_p_value < T,]
leads.d52rot.sign = leads.day52rot[leads.day52rot$global_corr_p_value < T,]

In [111]:
c(nrow(leads.d30.sign), nrow(leads.d52.sign), nrow(leads.d52rot.sign))

[1]  875 1027  460

In [112]:
RankAndKeepTop <- function(leads){
    leads1 = leads[order(leads$global_corr_p_value),]
    leads2 = leads1[-which(duplicated(leads1$feature)),]  
    leads2
}

In [113]:
t=0.05

In [114]:
### inner join significant sc day30 leads, all results from sc day52 leads
leads.d30.in.d52 = left_join(leads.d30.sign, res.day52, by = c("snp_id","feature_id"), 
                            suffix = c(".d30",".d52"))
# leads.d30.in.d52 = RankAndKeepTop(leads.d30.in.d52)

In [115]:
### inner join significant sc d30 leads in d52, all results from sc d52rot
leads.d30.in.d52.in.rot = left_join(leads.d30.in.d52, res.day52rot, by = c("snp_id","feature_id"))
# leads.d30.in.d52.in.rot = RankAndKeepTop(leads.d30.in.d52.in.rot)
d30 = leads.d30.in.d52.in.rot
d30$p_value.rot = d30$p_value
d30$beta.rot = d30$beta
d30 = d30[!(is.na(d30$p_value.d52)) & !(is.na(d30$p_value.rot)),]
nrow(d30[(d30$p_value.d52 >= t) & (d30$p_value.rot >= t),])

[1] 42

In [116]:
### inner join significant sc day52 leads, all results from sc day30 leads
leads.d52.in.d30 = left_join(leads.d52.sign, res.day30, by = c("snp_id","feature_id"), 
                            suffix = c(".d52",".d30"))
# leads.d52.in.d30 = RankAndKeepTop(leads.d52.in.d30)

In [117]:
### inner join significant sc day52 leads in day30, all results from sc d52rot
leads.d52.in.d30.in.rot = left_join(leads.d52.in.d30, res.day52rot, by = c("snp_id","feature_id"))
# leads.d52.in.d30.in.rot = RankAndKeepTop(leads.d52.in.d30.in.rot)
d52 = leads.d52.in.d30.in.rot
d52$p_value.rot = d52$p_value
d52$beta.rot = d52$beta
d52 = d52[!(is.na(d52$p_value.d30)) & !(is.na(d52$p_value.rot)),]
nrow(d52[(d52$p_value.d30 >= t) & (d52$p_value.rot >= t),])

[1] 30

In [118]:
### inner join significant sc day52-ROT leads, all results from sc day30 leads
leads.rot.in.d30 = left_join(leads.d52rot.sign, res.day30, by = c("snp_id","feature_id"), 
                            suffix = c(".rot",".d30"))
# leads.rot.in.d30 = RankAndKeepTop(leads.rot.in.d30)

In [119]:
# ### inner join significant sc day52-ROT leads in day30, all results from sc day52
leads.rot.in.d30.in.d52 = left_join(leads.rot.in.d30, res.day52, by = c("snp_id","feature_id"))
# leads.rot.in.d30.in.d52 = RankAndKeepTop(leads.rot.in.d30.in.d52)
rot = leads.rot.in.d30.in.d52
rot$p_value.d52 = rot$p_value
rot$beta.d52 = rot$beta
rot = rot[!(is.na(rot$p_value.d30)) & !(is.na(rot$p_value.d52)),]
nrow(rot[(rot$p_value.d30 >= t) & (rot$p_value.d52 >= t),])

[1] 17

In [120]:
#load map HGNC to ensembl
mapping_df = read.csv('/nfs/leia/research/stegle/dseaton/genomes/hg19/annotation/geneid_mappings/hgnc_symbol2ensembl_gene_id.txt', sep='\t')
colnames(mapping_df) = c("gene_name","feature_id")
head(mapping_df,2)

gene_name,feature_id
SLC25A26,ENSG00000261657
,ENSG00000223116


In [121]:
d30_specific = d30[(d30$p_value.d52 >= t) & (d30$p_value.rot >= t),]
### convert ensembl_gene_ids into gene names
d30_specific = left_join(d30_specific, mapping_df) 
d30_specific$eqtl = paste0(d30_specific$gene_name,"-",d30_specific$snp_id)
# head(d30_specific,2)
d30_eqtls = unique(d30_specific$eqtl)
length(d30_eqtls)
head(d30_eqtls,2)

Joining, by = "feature_id"


[1] 42

[1] "LRTM1-3_54949415_C_T"  "BANF1-11_65750617_C_G"

In [122]:
d52_specific = d52[(d52$p_value.d30 >= t) & (d52$p_value.rot >= t),]
d52_specific = left_join(d52_specific, mapping_df)
d52_specific$eqtl = paste0(d52_specific$gene_name,"-",d52_specific$snp_id)
# head(d52_specific,2)
d52_eqtls = unique(d52_specific$eqtl)
length(d52_eqtls)
head(d52_eqtls,2)

Joining, by = "feature_id"


[1] 30

[1] "NRP2-2_206356524_A_G"  "KPNB1-17_45818550_T_C"

In [123]:
rot_specific = rot[(rot$p_value.d30 >= t) & (rot$p_value.d52 >= t),]
rot_specific = left_join(rot_specific, mapping_df) 
rot_specific$eqtl = paste0(rot_specific$gene_name,"-",rot_specific$snp_id)
# head(rot_specific,2)
rot_eqtls = unique(rot_specific$eqtl)
length(rot_eqtls)
head(rot_eqtls,2)

Joining, by = "feature_id"


[1] 17

[1] "C5orf45-5_179253926_A_G" "EIF2AK3-2_88829687_G_A"

In [124]:
#### 
df = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/scaled_absolute_eff_sizes_20mofas_as_spatial_downsample_50pct_cluster_labels.csv", row.names = 1)
df$eqtl = gsub("_-","-",df$gene)
head(df,2)

,label,gene,pp0,pp1,pp2,pp3,pp4,pp5,pp6,pp7,pp8,pp9,pp10,pp11,eqtl
0,4,ABHD10_-3_111697439_T_G,0,0,0,0,1,0,0,0,0,0,0,0,ABHD10-3_111697439_T_G
1,5,ABHD12B_-14_51328222_C_T,0,0,0,0,0,1,0,0,0,0,0,0,ABHD12B-14_51328222_C_T


In [125]:
df0 = df[,c("label","eqtl")]
df0$specific = "no"
df0[df0$eqtl %in% d30_eqtls,"specific"] = "d30"
df0[df0$eqtl %in% d52_eqtls,"specific"] = "d52"
df0[df0$eqtl %in% rot_eqtls,"specific"] = "d52_rot"
head(df0)

,label,eqtl,specific
0,4,ABHD10-3_111697439_T_G,no
1,5,ABHD12B-14_51328222_C_T,no
2,2,ACKR2-3_42817129_C_T,no
3,5,ADD2-2_70894426_A_T,no
4,4,ADSL-22_40757228_A_C,no
5,4,AIM1L-1_26673825_C_T,no


In [126]:
table(df0$specific)


    d30     d52 d52_rot      no 
      1       1       1     209 

In [127]:
table(df0$label)


 0  1  2  3  4  5  6  7  8  9 10 11 
 2  2 41 18 39 43  9 12 10 15 13  8 

In [128]:
table(df0$specific, df0$label)

         
           0  1  2  3  4  5  6  7  8  9 10 11
  d30      0  0  1  0  0  0  0  0  0  0  0  0
  d52      0  0  0  0  1  0  0  0  0  0  0  0
  d52_rot  0  0  0  0  0  0  0  1  0  0  0  0
  no       2  2 40 18 38 43  9 11 10 15 13  8